# Despliegue en la nube

En la documentación de FastAPI [hay un tutorial](https://fastapi.tiangolo.com/deployment/deta/) que me ha llamado la atención, consiste en desplegar una API en un servicio llamado [Deta.sh](https://www.deta.sh/?ref=fastapi). Esta plataforma es a priori más simple que `Heroku` así que vamos a probar, nunca está de más conocer otras alternativas.


## Cuenta e instalación

Lo que necesitamos de antemano es crear una cuenta en https://web.deta.sh. Una vez registrados, si iniciamos la sesión estaremos en nuestro *dashboard* que se verá más o menos así:

![](docs/img12.png)

Según el tutorial para desplegar la aplicación necesitamos instalar un programa llamado `deta` en la terminal de nuestro sistema operativo (revisad [la web del tutorial](https://fastapi.tiangolo.com/deployment/deta/), puede ser que esto cambie):

En **Mac** y **Linux**:

```bash
$ curl -fsSL https://get.deta.dev/cli.sh | sh
```

En **Windows** desde la *Powershell*:

```bash
$ iwr https://get.deta.dev/cli.ps1 -useb | iex
```

Reiniciamos la terminal y deberíamos ver algo así al ejecutar `deta version`:

![](docs/img13.png)

## Preparando el proyecto

Básicamente hay dos requisitos para desplegar el proyecto:

* Un fichero `main.py` desde donde se ejecutará la API.
* Un `requirements.txt` con las dependencias.

Así que tendremos que cambiar el nombre de `api.py` a `main.py`, cosas de la vida... 

El `requirements.txt` solo requiere dos dependencias para funcionar:

```toml
fastapi==0.79.0
pydantic==1.9.2
```

Fácil.

## Desplegando la API

A continuación ya con una nueva terminal abierta en el directorio del `gestor` nos identificamos en la plataforma:

```bash
$ deta login
```

Aquí me ha molestado que solo se permita Firefox o Chrome para identificarse, así que me ha tocado instalar Firefox:

```
Please, log in from the web page. Waiting...
https://web.deta.sh/cli/50581
Logged in successfully.
```

Una vez identificados creamos la aplicación, a la que ellos llaman `micro`:

```bash
$ deta new
```

Automáticamente se instalarán las dependencias y se subirá el proyecto:

```json
Successfully created a new micro
{
    "name": "gestor",
    "id": "f51339a1-b917-4338-9115-f8be8683315f",
    "project": "a08h6epi",
    "runtime": "python3.9",
    "endpoint": "https://vlavj0.deta.dev",
    "region": "eu-central-1",
    "visor": "disabled",
    "http_auth": "disabled"
}
```

En principio ya podemos acceder a la API que encontramos en https://vlavj0.deta.dev y  hacer pruebas en los `docs` dinámicos https://vlavj0.deta.dev/docs:

![](docs/img14.png)

## Problema con la persistencia

Una vez más, de forma similar a lo que ocurria con `Heroku`, la persistencia en ficheros no está soportada por `Deta` a no ser que se utilice su propia API, esa es la razón por la que si creamos un cliente, lo actualizamos o lo borramos es posible que se devuelva un error 500 (no se puede escribir en `clientes.csv`).

Para que la API sepa que se está ejecutando en `Deta` vamos a decirle al servicio que incluya una variable de entorno en un fichero `.env`:

`.env`

```
PROD=True
```

Y lo configuramos en el *micro*:

```bash
$ deta update -e .env
```

```
Updating environment variables...
Successfully updated micro's environment variables
```

En nuestro código haremos una pequeña comprobación, solo guardaremos el fichero en caso de que la variable de entorno `PROD` no exista:

`database.py`

```python
import os

@staticmethod
def guardar():
    if not "PROD" in os.environ:  # <-----
        with open(config.DATABASE_PATH, 'w', newline='\n') as fichero:
            writer = csv.writer(fichero, delimiter=';')
            for cliente in Clientes.lista:
                writer.writerow((cliente.dni, cliente.nombre, cliente.apellido))
```

Ahora desplegamos los cambios del proyecto:

```bash
$ deta deploy
```

```
Deploying...
Successfully deployed changes
```

Con esto ya no dará errores y la API funcionará de forma normal, teniendo en cuenta que si en algún momento desplegamos de nuevo el servicio, o se apaga por inactividiad, los datos volverán a su estado inicial.

Para más información sobre los comandos disponibles en `Deta` aquí os dejo la [documentación oficial](https://docs.deta.sh/docs/cli/commands/).